# Text to SQL Experiment

Recurrent neural networks (RNNs) are designed to solve sequence modelling problems like translations, because they process information incrementally and can maintain an internal state that uses past information to inform predictions.

A Long Short-Term Memory (LSTM) network is a type of recurrent neural network that has LSTM cell blocks instead of typical neural network layers. The main components of these cells are the input gate, the forget gate and the output gate. This architecture solves the <a href="https://en.wikipedia.org/wiki/Vanishing_gradient_problem">vanishing gradient problem</a>.

<img src='static/LSTM.png' />

<a href="http://adventuresinmachinelearning.com/recurrent-neural-networks-lstm-tutorial-tensorflow/">Source</a>

Below is an attempt to build and train a bi-directional Long Short-Term Memory (LSTM) model for mapping text to SQL. This baseline approach works by identifying an SQL template for the question, then predicting appropriate words to fill slots in the template.

## Data Exploration

This dataset contains natural language questions and corresponding SQL queries about course information at the University of Michigan.
Some questions were collected from the department Facebook page and others were written by students familiar with the database, instructed to write questions they might ask an academic advisor. The authors of the dataset manually labeled the questions with SQL. 

In [1]:
import pandas as pd
import numpy as np

import dynet as dy
from collections import Counter
from tqdm import tqdm
import pickle
import random
import json
import sys

In [2]:
advising = pd.read_json('txt2sql-data/advising.json')
advising.head()

,query-split,sentences,sql,variables
0,test,"[{'question-split': 'train', 'text': 'Can unde...",[SELECT DISTINCT COURSEalias0.ADVISORY_REQUIRE...,"[{'example': 'EECS', 'location': 'sql-only', '..."
1,train,"[{'question-split': 'test', 'text': 'What 's t...","[SELECT DISTINCT COURSEalias0.DEPARTMENT , COU...","[{'example': 'MDE', 'location': 'both', 'name'..."
2,test,"[{'question-split': 'test', 'text': 'Can I tak...",[SELECT COUNT( * ) > 0 FROM COURSE AS COURSEal...,"[{'example': '370', 'location': 'both', 'name'..."
3,test,"[{'question-split': 'test', 'text': 'Can you s...","[SELECT DISTINCT COURSEalias0.DEPARTMENT , COU...","[{'example': 'EECS', 'location': 'both', 'name..."
4,test,"[{'question-split': 'train', 'text': 'Who usua...","[SELECT COUNT( INSTRUCTORalias0.NAME ) , INSTR...","[{'example': 'EECS', 'location': 'both', 'name..."


In [3]:
print('Sample Question:')
advising['sentences'][2][1]

Sample Question:


{'question-split': 'test',
 'text': 'Am I able to take department0 number0 and department0 number1 in the same semester ?',
 'variables': {'department0': 'RCIDIV', 'number0': '316', 'number1': '224'}}

In [4]:
print('Sample SQL:')
advising['sql'][2]

Sample SQL:


['SELECT COUNT( * ) > 0 FROM COURSE AS COURSEalias0 , COURSE AS COURSEalias1 , COURSE_OFFERING AS COURSE_OFFERINGalias0 , COURSE_OFFERING AS COURSE_OFFERINGalias1 WHERE COURSE_OFFERINGalias1.SEMESTER = COURSE_OFFERINGalias0.SEMESTER AND COURSEalias0.COURSE_ID = COURSE_OFFERINGalias0.COURSE_ID AND COURSEalias0.DEPARTMENT = "department0" AND COURSEalias0.NUMBER = number0 AND COURSEalias1.COURSE_ID = COURSE_OFFERINGalias1.COURSE_ID AND COURSEalias1.DEPARTMENT = "department0" AND COURSEalias1.NUMBER = number1 AND ( NOT ( ( COURSEalias1.COURSE_ID = COURSE_PREREQUISITEalias0.COURSE_ID AND COURSEalias0.COURSE_ID = COURSE_PREREQUISITEalias0.PRE_COURSE_ID ) OR ( COURSEaliasr02.COURSE_ID = COURSE_PREREQUISITEalias0.PRE_COURSE_ID AND COURSEalias1.COURSE_ID = COURSE_PREREQUISITEalias0.COURSE_ID ) ) ) ;']

## Data Preprocessing

The first step of the process is to identify the variable names, SQL template, and complete sql statements for all the data in the dataset.

In [5]:
def insert_variables(sql, sql_variables, sent, sent_variables):
    tokens = []
    tags = []
    seen_sent_variables = set()
    for token in sent.strip().split():
        if (token not in sent_variables):
            tokens.append(token)
            tags.append("O")
        else:
            assert len(sent_variables[token]) > 0
            seen_sent_variables.add(token)
            for word in sent_variables[token].split():
                tokens.append(word)
                tags.append(token)

    sql_tokens = []
    for token in sql.strip().split():
        if token.startswith('"%') or token.startswith("'%"):
            sql_tokens.append(token[:2])
            token = token[2:]
        elif token.startswith('"') or token.startswith("'"):
            sql_tokens.append(token[0])
            token = token[1:]

        if token.endswith('%"') or token.endswith("%'"):
            sql_tokens.append(token[:-2])
            sql_tokens.append(token[-2:])
        elif token.endswith('"') or token.endswith("'"):
            sql_tokens.append(token[:-1])
            sql_tokens.append(token[-1])
        else:
            sql_tokens.append(token)

    template = []
    complete = []
    for token in sql_tokens:
        # Do the template
        if token in seen_sent_variables:
            # The token is a variable name that will be copied from the sentence
            template.append(token)
        elif (token not in sent_variables) and (token not in sql_variables):
            # The token is an SQL keyword
            template.append(token)
        elif token in sent_variables and sent_variables[token] != '':
            # The token is a variable whose value is unique to this questions,
            # but is not explicitly given
            template.append(sent_variables[token])
        else:
            # The token is a variable whose value is not unique to this
            # question and not explicitly given
            template.append(sql_variables[token])

        # Do the complete case
        if token in sent_variables and sent_variables[token] != '':
            complete.append(sent_variables[token])
        elif token in sql_variables:
            complete.append(sql_variables[token])
        else:
            complete.append(token)

    return (tokens, tags, ' '.join(template), ' '.join(complete))

In [6]:
def get_tagged_data_for_query(data):
    # By default, set to the query split value
    dataset = data['query-split']
    # split can be adjusted for small datasets that require cross-validation
    split = None
    use_all_sql = True
    
    if split is not None:
        if str(split) == str(dataset):
            dataset = "test"
        else:
            dataset = "train"

    for sent_info in data['sentences']:
        # For question split, set to this question's value
        dataset = sent_info['question-split']
        if split is not None:
            if str(split) == str(dataset):
                dataset = "test"
            else:
                dataset = "train"

        for sql in data['sql']:
            sql_vars = {}
            for sql_var in data['variables']:
                sql_vars[sql_var['name']] = sql_var['example']
            text = sent_info['text']
            text_vars = sent_info['variables']

            yield (dataset, insert_variables(sql, sql_vars, text, text_vars))

            if not use_all_sql:
                break

# Building Vocabulary


The words that make up the sentences and SQL statements need to be mapped to integers in order to be fed to the model. This vocabulary is built using a dictionary with keys as integers and words as values.

In [7]:
class Vocab:
    def __init__(self, w2i):
        self.w2i = dict(w2i)
        self.i2w = {i:w for w,i in w2i.items()}

    @classmethod
    def from_corpus(cls, corpus):
        w2i = {}
        for word in corpus:
            w2i.setdefault(word, len(w2i))
        return Vocab(w2i)

    def size(self):
        return len(self.w2i.keys())

def build_vocab(sentences):
    counts = Counter()
    words = {"<UNK>"}
    tag_set = set()
    template_set = set()
    for tokens, tags, template, complete in train:
        template_set.add(template)
        for tag in tags:
            tag_set.add(tag)
        for token in tokens:
            counts[token] += 1

    for word in counts:
        if counts[word] > 0:
            words.add(word)

    vocab_tags = Vocab.from_corpus(tag_set)
    vocab_words = Vocab.from_corpus(words)
    vocab_templates = Vocab.from_corpus(template_set)

    return vocab_words, vocab_tags, vocab_templates

## Input Data Pipeline

Next, the dataset is split into a training set and a test set.

In [8]:
train = []
dev = []
test = []
with open('txt2sql-data/advising.json') as input_file:
    data = json.load(input_file)
    for example in data:
        for dataset, instance in get_tagged_data_for_query(example):
            if dataset == 'train':
                train.append(instance)
            elif dataset == 'dev':
                train.append(instance)
            elif dataset == 'test':
                test.append(instance)
            elif dataset == 'exclude':
                pass
            else:
                assert False, dataset

Example of data that comes out at the end of the preprocessing pipeline:

In [9]:
for tokens, tags, template, complete in train:
    print(tokens)
    print(tags)
    print(template)
    print(complete)
    break

['Can', 'undergrads', 'take', '550', '?']
['O', 'O', 'O', 'number0', 'O']
SELECT DISTINCT COURSEalias0.ADVISORY_REQUIREMENT , COURSEalias0.ENFORCED_REQUIREMENT , COURSEalias0.NAME FROM COURSE AS COURSEalias0 WHERE COURSEalias0.DEPARTMENT = " EECS " AND COURSEalias0.NUMBER = number0 ;
SELECT DISTINCT COURSEalias0.ADVISORY_REQUIREMENT , COURSEalias0.ENFORCED_REQUIREMENT , COURSEalias0.NAME FROM COURSE AS COURSEalias0 WHERE COURSEalias0.DEPARTMENT = " EECS " AND COURSEalias0.NUMBER = 550 ;


In [10]:
vocab_words, vocab_tags, vocab_templates = build_vocab(train)
UNK = vocab_words.w2i["<UNK>"]
NWORDS = vocab_words.size()
NTAGS = vocab_tags.size()
NTEMPLATES = vocab_templates.size()

print(f"Data Summary:")
print(f"{NTEMPLATES} templates", f"{NWORDS} words", f"{NTAGS} tags", f"{UNK} unknown words")

Data Summary:
219 templates 2591 words 23 tags 2537 unknown words


# Model 

The model was built using DyNet, a neural network library developed by Carnegie Mellon University and many others.
It is designed to run efficiently on CPUs or GPUs, and is very useful for natural language processing tasks.

In [11]:
model = dy.Model()

# This trainer performs stochastic gradient descent
# this is the most common optimization procedure for neural networks.
trainer = dy.SimpleSGDTrainer(model, learning_rate=0.1)

## Hyperparameters

# Input dimension 
DIM_WORD = 64

# Number of input layers
LSTM_LAYERS = 2

# Dimension of the recurrent units
DIM_HIDDEN_LSTM = 128

# Dimension of the hidden layers
DIM_HIDDEN_MLP = 32

# Dimension of the hidden layers that predict the template
DIM_HIDDEN_TEMPLATE = 32

# Word Embeddings
pEmbedding = model.add_lookup_parameters((NWORDS, DIM_WORD))

# Layers  
pHidden = model.add_parameters((DIM_HIDDEN_MLP, DIM_HIDDEN_LSTM*2))
pOutput = model.add_parameters((NTAGS, DIM_HIDDEN_MLP))

# This allows us to create a standard LSTM
builders = [
    dy.LSTMBuilder(LSTM_LAYERS, DIM_WORD, DIM_HIDDEN_LSTM, model),
    dy.LSTMBuilder(LSTM_LAYERS, DIM_WORD, DIM_HIDDEN_LSTM, model),
]

pHiddenTemplate = model.add_parameters((DIM_HIDDEN_TEMPLATE, DIM_HIDDEN_LSTM*2))
pOutputTemplate = model.add_parameters((NTEMPLATES, DIM_HIDDEN_TEMPLATE))

## Training the model

In [12]:
def build_tagging_graph(words, tags, template, builders, train=True):
    dy.renew_cg()
    
    dropout_rate = 0
    mlp = True

    if train and dropout_rate is not None and dropout_rate > 0:
        for b in builders:
            b.set_dropouts(dropout_rate, dropout_rate)

    f_init, b_init = [b.initial_state() for b in builders]

    wembs = [dy.lookup(pEmbedding, w) for w in words]
    if train: # Add noise in training as a regularizer
        wembs = [dy.noise(we, 0.1) for we in wembs]

    fw_states = [x for x in f_init.add_inputs(wembs)]
    bw_states = [x for x in b_init.add_inputs(reversed(wembs))]
    fw = [x.output() for x in fw_states]
    bw = [x.output() for x in bw_states]
    
    # Output of the input gate
    O = dy.parameter(pOutput)
    
    if mlp:
        # Output of the hidden layer
        H = dy.parameter(pHidden)
    errs = []
    pred_tags = []
    
    for f, b, t in zip(fw, reversed(bw), tags):
        # The current input, and output from the previous state are combined
        f_b = dy.concatenate([f,b])
        if mlp:
        # The previous LSTM cell outputs are squashed b/n -1 and 1
            f_b = dy.tanh(H * f_b)
        # squashed input is then multiplied element-wise by the output of the input gate. 
        r_t = O * f_b
        if train:
            err = dy.pickneglogsoftmax(r_t, t)
            errs.append(err)
        else:
            out = dy.softmax(r_t)
            chosen = np.argmax(out.npvalue())
            pred_tags.append(vocab_tags.i2w[chosen])

    O_template = dy.parameter(pOutputTemplate)
    H_template = dy.parameter(pHiddenTemplate)
    f_bt = dy.concatenate([fw_states[-1].s()[0], bw_states[-1].s()[0]])
    f_bt = dy.tanh(H_template * f_bt)
    r_tt = O_template * f_bt
    pred_template = None
    if train:
        err = dy.pickneglogsoftmax(r_tt, template)
        errs.append(err)
    else:
        out = dy.softmax(r_tt)
        chosen = np.argmax(out.npvalue())
        pred_template = vocab_templates.i2w[chosen]

    return pred_tags, pred_template, errs

In [13]:
def insert_tagged_tokens(tokens, tags, template):
    to_insert = {}
    cur = (None, [])
    for token, tag in zip(tokens, tags):
        if tag != cur[0]:
            if cur[0] is not None:
                value = ' '.join(cur[1])
                to_insert[cur[0]] = value
            if tag == 'O':
                cur = (None, [])
            else:
                cur = (tag, [token])
        else:
            cur[1].append(token)
    if cur[0] is not None:
        value = ' '.join(cur[1])
        to_insert[cur[0]] = value

    modified = []
    for token in template.split():
        modified.append(to_insert.get(token, token))

    return ' '.join(modified)

In [14]:
def run_eval(data, builders, iteration, step):
    if len(data) == 0:
        print("No data for eval")
        return -1
    correct_tags = 0.0
    total_tags = 0.0
    complete_match = 0.0
    templates_match = 0.0
    oracle = 0.0
    for tokens, tags, template, complete in data:
        word_ids = [vocab_words.w2i.get(word, UNK) for word in tokens]
        tag_ids = [0 for tag in tags]
        pred_tags, pred_template, _ = build_tagging_graph(word_ids, tag_ids, 0, builders, False)
        gold_tags = tags
        for gold, pred in zip(gold_tags, pred_tags):
            total_tags += 1
            if gold == pred: correct_tags += 1
        pred_complete = insert_tagged_tokens(tokens, pred_tags, pred_template)
        if pred_complete == complete:
            complete_match += 1
        if pred_template == template:
            templates_match += 1
        if template in vocab_templates.w2i:
            oracle += 1

    tok_acc = correct_tags / total_tags
    complete_acc = complete_match / len(data)
    template_acc = templates_match / len(data)
    oracle_acc = oracle / len(data)
    
    print(f"Evaluation {iteration} - {step} => Tagging Accuracy: {tok_acc:.2%}% Template Accuracy: {template_acc:.2%}%")
    print(f"Total Accuracy: {complete_acc:.2%}%")
    return complete_acc

## Evaluation 

In [15]:
tagged = 0
loss = 0
best_dev_acc = 0.0

# Number of examples to decode between logging
log_freq = 1000000
# Number of examples to decode between evaluation runs
eval_freq = 1000000

iters_since_best_updated = 0
iters_without_improvement = 5
steps = 0
iters = 50

#  Training in 50 epochs'
for iteration in tqdm(range(iters)):
    random.shuffle(train)
    for tokens, tags, template, complete in train:
        steps += 1

        # Convert to indices
        word_ids = [vocab_words.w2i.get(word, UNK) for word in tokens]
        tag_ids = [vocab_tags.w2i[tag] for tag in tags]
        template_id = vocab_templates.w2i[template]

        # Decode and update
        _, _, errs = build_tagging_graph(word_ids, tag_ids, template_id, builders)
        sum_errs = dy.esum(errs)
        loss += sum_errs.scalar_value()
        tagged += len(tag_ids)
        sum_errs.backward()
        trainer.update()

        # Log status
        if steps % log_freq == 0:
            trainer.status()
            print(f"TrainLoss {iteration}-{steps}: {loss / tagged}")
            loss = 0
            tagged = 0
            sys.stdout.flush()
        if steps % eval_freq == 0:
            acc = run_eval(dev, builders, iteration, steps)
            if best_dev_acc < acc:
                best_dev_acc = acc
                iters_since_best_updated = 0
                print("New best Accuracy!", acc)
            sys.stdout.flush()
    iters_since_best_updated += 1
    if iters > 0 and iters_since_best_updated > 50:
       print(f'Stopping at {steps} iterations as there have been {iters_without_improvement} iterations without improvement')
       break
        
run_eval(test, builders, "End", "test")

  0%|          | 0/50 [00:00<?, ?it/s]

The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.


100%|██████████| 50/50 [16:13<00:00, 20.25s/it]


Evaluation End - test => Tagging Accuracy: 98.98%% Template Accuracy: 78.40%%
Total Accuracy: 74.80%%


0.7479541734860884

## Challenges & Limitations

The most significant challenges were configuring the input data pipeline and ensuring that the words were being correctly tagged and encoded. It was also challenging to select appropriate hyperparameters that would ensure the model does not underperform or overfit. My first iteration of the model had an accuracy of ~40%.

### Preprocessing
The inputs to the model was a dictionary that mapped words to integers, with no meaningful relationships between the words. Word embeddings are dense, floating-point vectors that are a more powerful way to encode vocabulary. Word embeddings are used to map human language into a geometric space, so that geometric relationships between word vectors reflect the semantic relationships between words. This could have been used to improve model performance by enabling it to better understand context.

### Building the model

The performance of the model could also have been enhanced by using mechanisms such as Attention, beam search, and copying. Attention helps the network pay attention to specific words in the input sequence, such as table column names. Beam Search will enable the decoder to predict the most probable next word, by taking into account the probability each word at every step in the sequence. Copying allows the model to generate words by simply copying them from the input sequence. These mechanism will enable the model to perform well on query-based splits.

### Tuning the model

Dropout is a regularization method that prevents activation and weight updates for some input and recurrent connections to LSTM units during training. The dropout rate could've been adjusted to possibly improve the model's performance.

## Sources

@InProceedings{data-sql-advising,
  author    = {Catherine Finegan-Dollak, Jonathan K. Kummerfeld, Li Zhang, Karthik Ramanathan, Sesh Sadasivam, Rui Zhang, and Dragomir Radev},
  title     = {Improving Text-to-SQL Evaluation Methodology},
  booktitle = {Proceedings of the 56th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers)},
  month     = {July},
  year      = {2018},
  address   = {Melbourne, Victoria, Australia},
  pages     = {351--360},
  url       = {http://aclweb.org/anthology/P18-1033},
}

This code is a modified version of the example [tagger code](https://github.com/clab/dynet/blob/master/examples/tagger/bilstmtagger.py) from the DyNet repository.

It is available under an Apache 2 license.